In [ ]:
##General imports
import csv
import os
import time
from datetime import datetime
import shutil
import copy
import numpy as np
from functools import partial
import platform

##Torch imports
import torch.nn.functional as F
import torch
from torch_geometric.data import DataLoader, Dataset
from torch_geometric.nn import DataParallel
import torch_geometric.transforms as T
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel
import torch.distributed as dist
import torch.multiprocessing as mp
from utils_train import evaluate,write_results,model_summary
from data import loader_setup,get_dataset,StructureDataset,GetY
training_parameters = { 
    "target_index": 0
    ,"loss": "mse_loss"       
}
job_parameters= { 
        "reprocess":"False"
        ,"job_name": "my_train_job"   
        ,"load_model": "False"
        ,"save_model": "True"
        ,"model_path": "my_model_shear2.pth"
        ,"write_output": "True"
            ,"parallel": "True"}
cwd = os.getcwd()
database= get_dataset(cwd,0)
(
    train_loader,
    val_loader,
    test_loader,
    train_sampler,
    train_dataset,
    _,
    _,
) = loader_setup(
    0,
    0,
    1,
    320,
    database,
    'cuda',
    42,
    0,
)
saved_model = torch.load(job_parameters["model_path"], map_location=torch.device("cuda")
    )
model = saved_model["full_model"]
model = model.to(rank)
model_summary(model)

##Get predictions
time_start = time.time()
test_error, test_out = evaluate(test_loader, model, training_parameters["loss"], rank, out=True)
elapsed_time = time.time() - time_start

print("Evaluation time (s): {:.5f}".format(elapsed_time))

print(test_error)


write_results(
        test_out, str(job_parameters["job_name"]) + "whole_bulk.csv"
    )

In [2]:
%run C:/Users/mom19004/Downloads/sams/main.py --data_dir "C:/Users/mom19004/Downloads/sams/" --data "M.pt"

train length: 9538 val length: 523 test length: 1046 unused length: 2 seed: 42


C:\Users\mom19004\Anaconda3\envs\pytorch\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


IndexError: too many indices for tensor of dimension 1